# Making predictions for new locations

For convenience, this notebook contains everything you need to make predictions using the saved model. You'll need a Google Earth Engine acount to download the inputs, but besides that all you need is some locations of interest. 

The first sections ets things up, the second is the fun bit where you'll download the data and make preditcions

# Setup

In [1]:
import ee
# ee.Authenticate() # Run once to link your EE account
ee.Initialize() # Run each session

In [2]:
from tqdm import tqdm # Progress bar
from datetime import timedelta

# A utility function to pull data for a set of locations
def sample(im, prop, lats, lons, scale=5000, reducer=ee.Reducer.first(), tileScale=4):
    points = []
    for lat, lon in zip(lats, lons):
        xy = ee.Geometry.Point([lon, lat])
        points.append(xy.buffer(scale))
    vals = im.reduceRegions(collection=ee.FeatureCollection(points), scale=scale, reducer=reducer, tileScale=tileScale).getInfo()
    if prop == '':
        return [v['properties'] for v in vals['features']]
    return [v['properties'][prop] for v in vals['features']]


def add_static_vars(df, scale=5000):
    lights = ee.ImageCollection("NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4").filter(ee.Filter.date('2010-01-01', '2018-03-08')).first()
    pop = ee.ImageCollection("CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density").filter(ee.Filter.date('2010-01-01', '2018-03-08')).first()
    ims = [lights, pop]

    for im in tqdm(ims):
        for i, reducer in enumerate([ee.Reducer.mean(), ee.Reducer.min(), ee.Reducer.max()]):
            sampled_values = sample(im, '', df['Lat'].values, df['Long'].values, reducer=reducer, scale=scale)
            for k in sampled_values[0].keys():
                arr = ['mean', 'min', 'max']
                df[k+'_'+str(scale)+'_' + arr[i]] = [sv[k] if k in sv.keys() else None for sv in sampled_values]
                if k == arr[i]:
                    df = df.rename(columns={k+'_'+str(scale)+'_' + arr[i]:'pop_density2010'+'_'+str(scale)+'_' + arr[i]})
    return df

# Image Collections
gfs = ee.ImageCollection("NOAA/GFS0P25") # Weather data

S5p_collections = {} # Sentinel 5p data, which comes in multiple collections
for COL in ['L3_NO2', 'L3_O3', 'L3_CO', 'L3_HCHO', 'L3_AER_AI', 'L3_SO2', 'L3_CH4', 'L3_CLOUD']: # 
    S5p_collections[COL] = ee.ImageCollection('COPERNICUS/S5P/OFFL/'+COL).map(lambda image: image.addBands(image.metadata('system:time_start')))
# Properties for each image we want to keep
s5p_props = {
    'L3_NO2':['NO2_column_number_density', 'tropospheric_NO2_column_number_density', 'stratospheric_NO2_column_number_density', 'NO2_slant_column_number_density', 'tropopause_pressure', 'absorbing_aerosol_index'],
    'L3_O3':['O3_column_number_density', 'O3_effective_temperature'],
    'L3_CO':['CO_column_number_density', 'H2O_column_number_density', 'cloud_height'],
    'L3_HCHO':['tropospheric_HCHO_column_number_density', 'tropospheric_HCHO_column_number_density_amf', 'HCHO_slant_column_number_density'],
    'L3_CLOUD':['cloud_fraction', 'cloud_top_pressure', 'cloud_top_height', 'cloud_base_pressure', 'cloud_base_height', 'cloud_optical_depth', 'surface_albedo'],
    'L3_AER_AI':['absorbing_aerosol_index', 'sensor_altitude', 'sensor_azimuth_angle', 'sensor_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle']
}

def add_timeseries(df, dates, reducer=ee.Reducer.first()):
    # Prepare dataframe with date x city
    date_col = []
    city_col = []
    for d in dates:
        for c in df.City.unique():
            date_col.append(d)
            city_col.append(c)

    data = pd.DataFrame({
        'Date':date_col,
        'City':city_col
    })
    data = pd.merge(data, df[['City', 'Lat', 'Long']], how='left', on='City')
    
    for d in tqdm(dates):
        # Weather is easy - a single image from the right date
        weather_image = gfs.filter(ee.Filter.date(str(d.date()), str((d+timedelta(days=1)).date()))).first() # Filter to get the relevant image
        # For the sentinel data, we get images from each collection and merge them
        s5p_images = []
        for COL in ['L3_NO2', 'L3_O3', 'L3_CO', 'L3_HCHO', 'L3_CLOUD', 'L3_AER_AI']:
            collection = S5p_collections[COL].filter(ee.Filter.date(str((d-timedelta(days=5)).date()), str(d.date())))
            image = collection.qualityMosaic('system:time_start') # The most recent image
            image = image.select(s5p_props[COL])
            s5p_images.append(image)
        s5p_image = ee.ImageCollection(s5p_images).toBands() # Merge into one image
    
        # Sample the weather data
        samples = sample(weather_image, '', df['Lat'].values, df['Long'].values, reducer=reducer)
        for prop in samples[0].keys():
            data.loc[data.Date==d, prop] = [p[prop] for p in samples]
            
        # Sample the sentinel data
        samples = sample(s5p_image, '', df['Lat'].values, df['Long'].values)
        for prop in samples[0].keys():
            data.loc[data.Date==d, prop] = [p[prop] for p in samples]
            
    return data

# Now the fun bit

In [3]:
# Load your locations to match this format:
import pandas as pd
cities = pd.DataFrame({
    'City':['Harare', 'Lusaka'],
    'Lat':[-17.8,-15.38],
    'Long':[31.08, 28.32]
})
cities.head()

,City,Lat,Long
0,Harare,-17.80,31.08
1,Lusaka,-15.38,28.32


In [4]:
# Specify dates
dates = pd.date_range('2020-01-01', '2020-01-04', freq='1D')

In [5]:
# Add static vars
cities_w_static = add_static_vars(cities.copy())

100%|██████████| 2/2 [00:05<00:00,  2.75s/it]


In [6]:
# Add timeseries
ts = add_timeseries(cities.copy(), dates, reducer=ee.Reducer.mean())
ts.head()

100%|██████████| 4/4 [00:38<00:00,  9.53s/it]


,Date,City,Lat,Long,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,...,4_cloud_optical_depth,4_cloud_top_height,4_cloud_top_pressure,4_surface_albedo,5_absorbing_aerosol_index,5_sensor_altitude,5_sensor_azimuth_angle,5_sensor_zenith_angle,5_solar_azimuth_angle,5_solar_zenith_angle
0,2020-01-01,Harare,-17.80,31.08,30.284542,64.517152,0.011835,21.128701,-1.347129,-0.675195,...,12.356079,6668.825427,44889.590843,0.380872,-1.020571,834491.262154,-101.169050,53.922629,-105.293815,31.669133
1,2020-01-01,Lusaka,-15.38,28.32,40.770137,87.502450,0.014240,19.661851,-0.313695,-1.581398,...,43.644369,5242.540284,54053.562815,0.484633,-0.257957,833600.522764,-100.959826,45.132484,-109.703873,30.000848
2,2020-01-02,Harare,-17.80,31.08,32.733817,80.631400,0.013801,20.057612,0.186643,-0.974955,...,16.174312,3933.434523,63476.461094,0.329794,-1.022922,834273.242731,-101.810167,30.910009,-105.842195,27.221785
3,2020-01-02,Lusaka,-15.38,28.32,34.187392,91.571483,0.014585,19.315761,-0.573623,-1.470822,...,5.680188,1479.899001,84954.764225,0.317455,-0.838425,833380.540503,-102.254185,15.757231,-111.351888,25.634975
4,2020-01-03,Harare,-17.80,31.08,33.018359,79.569327,0.013744,20.175666,-2.481068,-2.688929,...,14.537306,4550.288964,58918.127998,0.292986,-0.291448,833977.167034,77.050683,4.456613,-106.953025,22.816142


In [7]:
data = pd.merge(ts, cities_w_static, on=['City', 'Lat', 'Long'])
data.head()

,Date,City,Lat,Long,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,...,5_solar_zenith_angle,avg_vis_5000_mean,cf_cvg_5000_mean,avg_vis_5000_min,cf_cvg_5000_min,avg_vis_5000_max,cf_cvg_5000_max,pop_density2010_5000_mean,pop_density2010_5000_min,pop_density2010_5000_max
0,2020-01-01,Harare,-17.80,31.08,30.284542,64.517152,0.011835,21.128701,-1.347129,-0.675195,...,31.669133,60.495142,25.851762,46.892315,26,67.716476,26,2573.822051,2573.822021,2573.822021
1,2020-01-02,Harare,-17.80,31.08,32.733817,80.631400,0.013801,20.057612,0.186643,-0.974955,...,27.221785,60.495142,25.851762,46.892315,26,67.716476,26,2573.822051,2573.822021,2573.822021
2,2020-01-03,Harare,-17.80,31.08,33.018359,79.569327,0.013744,20.175666,-2.481068,-2.688929,...,22.816142,60.495142,25.851762,46.892315,26,67.716476,26,2573.822051,2573.822021,2573.822021
3,2020-01-04,Harare,-17.80,31.08,28.977537,89.543964,0.014138,18.726597,-2.367171,-1.849972,...,18.466113,60.495142,25.851762,46.892315,26,67.716476,26,2573.822051,2573.822021,2573.822021
4,2020-01-01,Lusaka,-15.38,28.32,40.770137,87.502450,0.014240,19.661851,-0.313695,-1.581398,...,30.000848,103.922143,63.516605,21.902353,43,147.594315,111,5384.729876,3392.412109,7093.277832


In [8]:
# Feature Engineering
to_lag = ['2_CO_column_number_density',
       '0_NO2_slant_column_number_density', '0_NO2_column_number_density',
       '3_tropospheric_HCHO_column_number_density', '0_tropopause_pressure',
       '3_HCHO_slant_column_number_density',
       'relative_humidity_2m_above_ground', '1_O3_column_number_density',
       '4_cloud_fraction', '0_stratospheric_NO2_column_number_density',
       '4_surface_albedo', '3_tropospheric_HCHO_column_number_density_amf',
       'u_component_of_wind_10m_above_ground', '4_cloud_optical_depth']

# Adding the same features
for shift in [1,2,3,5]:
    for col in to_lag:
        data[col+'shift'+str(shift)] = data.groupby(['City'])[col].transform(lambda x:x.shift(shift))
        data[col+'nshift'+str(shift)] = data.groupby(['City'])[col].transform(lambda x:x.shift(-shift))
for attr in ['day', 'month','year','week', 'dayofweek', 'weekofyear', 'days_in_month', 'is_month_start', 'is_month_end', 'dayofyear']:
    data[attr] = getattr(pd.DatetimeIndex(data['Date']), attr)
data['is_weekend'] = (data['dayofweek'] >= 5)*1
data['fortnight'] = data['day']%15
data['which_fortnight'] = data['day']//15

In [9]:
# Load the saved model
from catboost import CatBoostRegressor
model = CatBoostRegressor()
model.load_model('../Data/saved_models/catboost_01')

In [10]:
# Make predictions
preds = model.predict(data.drop(['Lat', 'Long', 'City', 'Date'], axis=1))
data['Predicted_PM25'] = preds

In [11]:
# View he result
data.head()

,Date,City,Lat,Long,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,...,dayofweek,weekofyear,days_in_month,is_month_start,is_month_end,dayofyear,is_weekend,fortnight,which_fortnight,Predicted_PM25
0,2020-01-01,Harare,-17.80,31.08,30.284542,64.517152,0.011835,21.128701,-1.347129,-0.675195,...,2,1,31,True,False,1,0,1,0,41.866902
1,2020-01-02,Harare,-17.80,31.08,32.733817,80.631400,0.013801,20.057612,0.186643,-0.974955,...,3,1,31,False,False,2,0,2,0,46.740090
2,2020-01-03,Harare,-17.80,31.08,33.018359,79.569327,0.013744,20.175666,-2.481068,-2.688929,...,4,1,31,False,False,3,0,3,0,26.274935
3,2020-01-04,Harare,-17.80,31.08,28.977537,89.543964,0.014138,18.726597,-2.367171,-1.849972,...,5,1,31,False,False,4,1,4,0,35.315149
4,2020-01-01,Lusaka,-15.38,28.32,40.770137,87.502450,0.014240,19.661851,-0.313695,-1.581398,...,2,1,31,True,False,1,0,1,0,29.584566


In [12]:
# In this example we can compate the two locations over the dates we specified. Both nice clean air apparently
data.groupby('City').mean()['Predicted_PM25']

City
Harare    37.549269
Lusaka    34.872906
Name: Predicted_PM25, dtype: float64

# Using this to make our final set of predictions

These three datasets were generated earlier. This section for records only.

In [18]:
def make_preds(ts, cities_w_static, savename):
    data = pd.merge(ts, cities_w_static, on=['City', 'Lat', 'Long'])
    # Feature Engineering
    to_lag = ['2_CO_column_number_density',
           '0_NO2_slant_column_number_density', '0_NO2_column_number_density',
           '3_tropospheric_HCHO_column_number_density', '0_tropopause_pressure',
           '3_HCHO_slant_column_number_density',
           'relative_humidity_2m_above_ground', '1_O3_column_number_density',
           '4_cloud_fraction', '0_stratospheric_NO2_column_number_density',
           '4_surface_albedo', '3_tropospheric_HCHO_column_number_density_amf',
           'u_component_of_wind_10m_above_ground', '4_cloud_optical_depth']

    # Adding the same features
    for shift in [1,2,3,5]:
        for col in to_lag:
            data[col+'shift'+str(shift)] = data.groupby(['City'])[col].transform(lambda x:x.shift(shift))
            data[col+'nshift'+str(shift)] = data.groupby(['City'])[col].transform(lambda x:x.shift(-shift))
    for attr in ['day', 'month','year','week', 'dayofweek', 'weekofyear', 'days_in_month', 'is_month_start', 'is_month_end', 'dayofyear']:
        data[attr] = getattr(pd.DatetimeIndex(data['Date']), attr)
    data['is_weekend'] = (data['dayofweek'] >= 5)*1
    data['fortnight'] = data['day']%15
    data['which_fortnight'] = data['day']//15

    model = CatBoostRegressor()
    model.load_model('../Data/saved_models/catboost_01')
    # Make predictions
    preds = model.predict(data.drop(['Lat', 'Long', 'City', 'Date'], axis=1))
    data['Predicted_PM25'] = preds
    data[['City', 'Date', 'Lat', 'Long', 'Predicted_PM25', 'pop_density2010_5000_max']].to_csv(savename, index=False)

In [19]:
ts = pd.read_csv('../Data/intermediate/za_cities_timeseries.csv')
static = pd.read_csv('../Data/intermediate/za_cities_w_static.csv')
make_preds(ts, static, '../Data/results/za_cities_predictions.csv')

In [25]:
ts = pd.read_csv('../Data/intermediate/af_cities_timeseries.csv')
static = pd.read_csv('../Data/intermediate/af_cities_w_static.csv').drop(['admin_name', 'capital', 'city_ascii', 'country', 'iso2', 'iso3', 'population', 'id'], axis=1)
make_preds(ts, static, '../Data/results/af_cities_predictions.csv')

In [26]:
ts = pd.read_csv('../Data/intermediate/pcs_timeseries.csv')
static = pd.read_csv('../Data/intermediate/pcs_w_static.csv')
make_preds(ts, static, '../Data/results/pcs_predictions.csv')